# Subsystem 1 — Allocation Notebook (Category → Product)
This notebook:
1. Loads trained **Subsystem 1 artifacts** (`subsystem1_artifacts.pkl`) containing:
   - `model` (sklearn Pipeline)
   - `threshold` (tuned decision threshold)
   - `feature_cols` (expected feature columns for inference)
2. Loads **raw transactional data** (your original dataset).
3. Builds a **latest feature snapshot** (per Category) using the most recent window.
4. Predicts `probability_up` per Category and derives `growth_signal`.
5. Computes **product share** per Category from recent history.
6. Allocates category-level forecast budget to **product-level recommended quantities**.

_Last updated: 2026-02-12 12:50_


In [1]:
# ==========================================
# 0) Imports & Configuration
# ==========================================
import os
import joblib
import numpy as np
import pandas as pd

from typing import Tuple

# ---- Files (update paths if needed) ----
ARTIFACTS_PATH = "subsystem1_artifacts.pkl"   # produced by training notebook
RAW_DATA_PATH  = "ML-Dataset.csv"             # your raw dataset

# ---- Windows (should match training assumptions) ----
FEATURE_WINDOW_DAYS = 28
SHARE_WINDOW_DAYS   = 90
FORECAST_HORIZON_DAYS = 28   # can also run 7/14/30; keep 28 for now

# ---- Allocation sensitivity ----
ALPHA = 0.5   # how strongly probability_up scales forecast budget

# Sanity
print("CWD:", os.getcwd())
print("Files:", [f for f in os.listdir() if f.endswith(('.pkl','.csv','.ipynb'))])


CWD: /Users/ayemyatmyathmwe/Documents/Murdoch Singapore/BIT_AIAS&BIS_Murdoch/TJA_2026/ICT304/Assignments
Files: ['Subsystem1.ipynb', 'ML-Dataset.csv', 'rolling_supervised_dataset.csv', 'SubSystem1_v1.ipynb', 'subsystem1_artifacts.pkl', 'sub1_category.ipynb', 'subsystem1_best_model.pkl', 'sub1_allocation_clean.ipynb', 'SubSystem_v2.ipynb', 'rolling_window_builder.ipynb', 'sub1_category_v2.ipynb']


## 1) Load Subsystem 1 artifacts (model + threshold + feature columns)

In [2]:
artifacts = joblib.load(ARTIFACTS_PATH)
model = artifacts["model"]
BEST_THRESHOLD = float(artifacts["threshold"])
FEATURE_COLS = list(artifacts.get("feature_cols", []))

print("Loaded artifacts:")
print(" - threshold:", BEST_THRESHOLD)
print(" - n_features expected:", len(FEATURE_COLS))
print(" - feature_cols:", FEATURE_COLS)


Loaded artifacts:
 - threshold: 0.09000000000000001
 - n_features expected: 0
 - feature_cols: []


## 2) Load raw data and basic cleaning

In [3]:
df_raw = pd.read_csv(RAW_DATA_PATH)

# Parse dates
df_raw["OrderDate"] = pd.to_datetime(df_raw["OrderDate"], errors="coerce")

# Basic filters (adjust if you want include Pending etc.)
if "Status" in df_raw.columns:
    df_raw = df_raw[df_raw["Status"].astype(str).str.lower().eq("shipped")]

# Required columns check
required = {"OrderDate","CategoryName","ProductName","OrderItemQuantity"}
missing = required - set(df_raw.columns)
if missing:
    raise ValueError(f"Missing required columns in raw dataset: {missing}")

# Ensure numeric quantity
df_raw["OrderItemQuantity"] = pd.to_numeric(df_raw["OrderItemQuantity"], errors="coerce").fillna(0)

print("Raw rows:", len(df_raw))
print("Date range:", df_raw["OrderDate"].min(), "→", df_raw["OrderDate"].max())
print("Categories:", df_raw["CategoryName"].nunique(), "Products:", df_raw["ProductName"].nunique())


Raw rows: 183
Date range: 2013-06-21 00:00:00 → 2017-11-01 00:00:00
Categories: 5 Products: 147


/var/folders/mb/7kg509493q3gkj2g2ktj03200000gn/T/ipykernel_90692/3599902713.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_raw["OrderDate"] = pd.to_datetime(df_raw["OrderDate"], errors="coerce")


## 3) Build latest category-level feature snapshot (matching training feature columns)
We compute features using the most recent `FEATURE_WINDOW_DAYS` days.

**Note**: This snapshot is for inference, not rolling dataset building.
We also add safe time features: `month`, `week_of_year`, `day_of_week` (from window_end).

In [4]:
def build_latest_category_features(
    df: pd.DataFrame,
    window_days: int = 28
) -> Tuple[pd.DataFrame, pd.Timestamp, pd.Timestamp]:
    """Return per-category features over the most recent window_days."""
    latest_date = df["OrderDate"].max().normalize()
    window_start = latest_date - pd.Timedelta(days=window_days-1)
    window_end = latest_date

    df_recent = df[(df["OrderDate"] >= window_start) & (df["OrderDate"] <= window_end)].copy()

    # Aggregate daily demand per category
    daily = (df_recent
             .groupby(["CategoryName", "OrderDate"])["OrderItemQuantity"]
             .sum()
             .reset_index())

    categories = daily["CategoryName"].unique()
    rows = []

    # Build a full daily index for each category to get correct zeros
    full_idx = pd.date_range(window_start, window_end, freq="D")

    for cat in categories:
        s = daily[daily["CategoryName"] == cat].set_index("OrderDate")["OrderItemQuantity"]
        s = s.reindex(full_idx, fill_value=0)

        cur_mean = float(s.mean())
        cur_std = float(s.std(ddof=0))  # population std
        coverage = float((s > 0).mean())
        volatility_ratio = float(cur_std / (cur_mean + 1e-6))

        # For inference, intra_growth may not be available; set to 0 (neutral)
        intra_growth = 0.0

        # Safe calendar features from window_end
        month = int(window_end.month)
        week_of_year = int(window_end.isocalendar().week)
        day_of_week = int(window_end.dayofweek)

        rows.append({
            "CategoryName": cat,
            "cur_mean": cur_mean,
            "cur_std": cur_std,
            "coverage": coverage,
            "volatility_ratio": volatility_ratio,
            "intra_growth": intra_growth,
            "month": month,
            "week_of_year": week_of_year,
            "day_of_week": day_of_week,
            "window_start": window_start,
            "window_end": window_end
        })

    feat = pd.DataFrame(rows)
    return feat, window_start, window_end

latest_feat, window_start, window_end = build_latest_category_features(df_raw, FEATURE_WINDOW_DAYS)
print("Latest feature snapshot window:", window_start.date(), "→", window_end.date())
latest_feat.head()


Latest feature snapshot window: 2017-10-05 → 2017-11-01


,CategoryName,cur_mean,cur_std,coverage,volatility_ratio,intra_growth,month,week_of_year,day_of_week,window_start,window_end
0,CPU,5.178571,26.908646,0.035714,5.196151,0.0,11,44,2,2017-10-05,2017-11-01
1,Mother Board,2.785714,14.474996,0.035714,5.196151,0.0,11,44,2,2017-10-05,2017-11-01
2,Storage,4.285714,22.269225,0.035714,5.196151,0.0,11,44,2,2017-10-05,2017-11-01
3,Video Card,6.857143,28.104034,0.071429,4.098504,0.0,11,44,2,2017-10-05,2017-11-01


## 4) Align features to training schema and predict category probabilities
We ensure the columns match what the model expects (`FEATURE_COLS`).

In [5]:
# If FEATURE_COLS wasn't saved, assume current columns are correct
if not FEATURE_COLS:
    FEATURE_COLS = [c for c in latest_feat.columns if c not in ["window_start","window_end"]]
    print("No feature_cols found in artifacts; using inferred FEATURE_COLS:", FEATURE_COLS)

# Build X_infer with exactly the expected columns (extra columns dropped, missing filled)
X_infer = latest_feat.copy()

# Remove meta date fields from inference features if present in FEATURE_COLS (shouldn't be)
# Keep only FEATURE_COLS in exact order
for col in FEATURE_COLS:
    if col not in X_infer.columns:
        X_infer[col] = 0

X_infer = X_infer[FEATURE_COLS]

# Predict
proba = model.predict_proba(X_infer)[:, 1]
latest_feat["probability_up"] = proba
latest_feat["growth_signal"] = (latest_feat["probability_up"] >= BEST_THRESHOLD).astype(int)

latest_feat[["CategoryName","probability_up","growth_signal","window_start","window_end"]].sort_values("probability_up", ascending=False)


No feature_cols found in artifacts; using inferred FEATURE_COLS: ['CategoryName', 'cur_mean', 'cur_std', 'coverage', 'volatility_ratio', 'intra_growth', 'month', 'week_of_year', 'day_of_week']


,CategoryName,probability_up,growth_signal,window_start,window_end
2,Storage,0.848593,1,2017-10-05,2017-11-01
1,Mother Board,0.846191,1,2017-10-05,2017-11-01
0,CPU,0.804415,1,2017-10-05,2017-11-01
3,Video Card,0.766360,1,2017-10-05,2017-11-01


## 5) Compute product share per category (last 90 days)
This gives the proportional allocation weights for each product within each category.

In [6]:
def compute_product_shares(df: pd.DataFrame, days: int = 90) -> pd.DataFrame:
    latest_date = df["OrderDate"].max().normalize()
    start = latest_date - pd.Timedelta(days=days-1)

    hist = df[(df["OrderDate"] >= start) & (df["OrderDate"] <= latest_date)].copy()

    prod_qty = (hist.groupby(["CategoryName","ProductName"])["OrderItemQuantity"]
                .sum()
                .reset_index()
                .rename(columns={"OrderItemQuantity":"product_qty"}))

    cat_qty = (prod_qty.groupby("CategoryName")["product_qty"]
               .sum()
               .reset_index()
               .rename(columns={"product_qty":"category_qty"}))

    out = prod_qty.merge(cat_qty, on="CategoryName", how="left")
    out["product_share"] = out["product_qty"] / (out["category_qty"] + 1e-9)
    return out, start, latest_date

shares, share_start, share_end = compute_product_shares(df_raw, SHARE_WINDOW_DAYS)
print("Share window:", share_start.date(), "→", share_end.date())
shares.sort_values(["CategoryName","product_share"], ascending=[True, False]).head(10)


Share window: 2017-08-04 → 2017-11-01


,CategoryName,ProductName,product_qty,category_qty,product_share
5,CPU,Intel Xeon E5-2695 V3 (OEM/Tray),148,698,0.212034
4,CPU,Intel Xeon E5-2683 V4,145,698,0.207736
1,CPU,Intel Xeon E5-2643 V3 (OEM/Tray),139,698,0.199140
3,CPU,Intel Xeon E5-2660 V4,118,698,0.169054
2,CPU,Intel Xeon E5-2650 V3 (OEM/Tray),104,698,0.148997
0,CPU,Intel Core i7-5960X (OEM/Tray),44,698,0.063037
6,Mother Board,ASRock C2750D4I,133,495,0.268687
7,Mother Board,ASRock X99 Extreme11,90,495,0.181818
10,Mother Board,Intel DG43RK,78,495,0.157576
8,Mother Board,Asus ROG STRIX X99 GAMING,73,495,0.147475


## 6) Allocate category forecast budget to products
We define a simple budget:

- `category_base_forecast = cur_mean × FORECAST_HORIZON_DAYS`
- `category_adjusted_forecast = category_base_forecast × (1 + ALPHA × probability_up)`

Then allocate:

- `recommended_qty = category_adjusted_forecast × product_share`

You can tune `ALPHA` later if needed.

In [8]:
# Category budget
latest_feat["category_base_forecast"] = latest_feat["cur_mean"] * FORECAST_HORIZON_DAYS
latest_feat["category_adjusted_forecast"] = latest_feat["category_base_forecast"] * (1 + ALPHA * latest_feat["probability_up"])

# Merge budget with shares
alloc = shares.merge(
    latest_feat[["CategoryName","probability_up","growth_signal","category_adjusted_forecast"]],
    on="CategoryName",
    how="left"
)

alloc["recommended_qty"] = (
    alloc["category_adjusted_forecast"] * alloc["product_share"]
).round(0).fillna(0).astype("Int64")

# Sort for readability
alloc_out = alloc[[
    "CategoryName","ProductName",
    "product_share","product_qty","category_qty",
    "probability_up","growth_signal",
    "category_adjusted_forecast","recommended_qty"
]].sort_values(["CategoryName","recommended_qty"], ascending=[True, False])

alloc_out.head(20)


,CategoryName,ProductName,product_share,product_qty,category_qty,probability_up,growth_signal,category_adjusted_forecast,recommended_qty
5,CPU,Intel Xeon E5-2695 V3 (OEM/Tray),0.212034,148,698,0.804415,1.0,203.320067,43
4,CPU,Intel Xeon E5-2683 V4,0.207736,145,698,0.804415,1.0,203.320067,42
1,CPU,Intel Xeon E5-2643 V3 (OEM/Tray),0.199140,139,698,0.804415,1.0,203.320067,40
3,CPU,Intel Xeon E5-2660 V4,0.169054,118,698,0.804415,1.0,203.320067,34
2,CPU,Intel Xeon E5-2650 V3 (OEM/Tray),0.148997,104,698,0.804415,1.0,203.320067,30
0,CPU,Intel Core i7-5960X (OEM/Tray),0.063037,44,698,0.804415,1.0,203.320067,13
6,Mother Board,ASRock C2750D4I,0.268687,133,495,0.846191,1.0,111.001433,30
7,Mother Board,ASRock X99 Extreme11,0.181818,90,495,0.846191,1.0,111.001433,20
10,Mother Board,Intel DG43RK,0.157576,78,495,0.846191,1.0,111.001433,17
8,Mother Board,Asus ROG STRIX X99 GAMING,0.147475,73,495,0.846191,1.0,111.001433,16


## 7) Export outputs (CSV)
Exports:
- `subsystem1_category_output.csv`
- `subsystem1_product_recommendations.csv`

In [9]:
category_out = latest_feat[[
    "CategoryName","probability_up","growth_signal",
    "category_base_forecast","category_adjusted_forecast",
    "window_start","window_end"
]].sort_values("probability_up", ascending=False)

category_out.to_csv("subsystem1_category_output.csv", index=False)
alloc_out.to_csv("subsystem1_product_recommendations.csv", index=False)

print("Saved:")
print(" - subsystem1_category_output.csv")
print(" - subsystem1_product_recommendations.csv")


Saved:
 - subsystem1_category_output.csv
 - subsystem1_product_recommendations.csv


## 8) Quick sanity checks 
- Do recommendations sum back to category budget?
- Are there categories with missing probability (should not happen)?

In [11]:
# Check sums close to category budget
check = (alloc_out.groupby("CategoryName")["recommended_qty"].sum()
         .reset_index()
         .merge(category_out[["CategoryName","category_adjusted_forecast"]], on="CategoryName", how="left"))

check["diff"] = check["recommended_qty"] - (
    check["category_adjusted_forecast"]
    .fillna(0)
    .round(0)
    .astype("Int64")
)
check.sort_values("diff", ascending=False).head(10)


,CategoryName,recommended_qty,category_adjusted_forecast,diff
2,RAM,0,NaN,0
3,Storage,171,170.915587,0
4,Video Card,266,265.570605,0
0,CPU,202,203.320067,-1
1,Mother Board,110,111.001433,-1
